In [1]:
import os
import os.path as op
import mne
import numpy as np
import re
import itertools
from typing import Tuple, Iterator
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch
from bids import BIDSLayout

In [2]:
BIDS_ROOT = '../data/bids'
STIM_FREQS = np.array([50, 100, 150, 200, 250])

In [26]:
KeyType = Tuple[str, str, str, str]

def get_fpaths(bids_root) -> Iterator[KeyType]:
    # Get filepaths
    layout = BIDSLayout(bids_root, derivatives = True)
    fpaths = layout.get(scope = 'preprocessing',
                        extension = 'fif.gz',
                        return_type = 'filename')
    fpaths.pop(0)
    
    # Get corresponding subject number
    filter_subs = re.compile('sub-(\d)_')
    subs = list(map(filter_subs.findall, fpaths))
    subs = list(itertools.chain(*subs))
    
    # Get corresponding run number
    filter_runs = re.compile('run-(\d)')
    runs = list(map(filter_runs.findall, fpaths))
    runs = list(itertools.chain(*runs))
    
    for i in range(len(fpaths)):
        key = (fpaths[i], subs[i], 'tasks', runs[i])
        yield key

def load_fif(fname):
    epochs = mne.read_epochs(fname)
    return epochs

def get_power_at_stim_freqs(epochs, stim_freqs):
    n_cycles = stim_freqs / 7.  # different number of cycle per frequency
    power = tfr_morlet(epochs, 
                       freqs = stim_freqs, 
                       n_cycles = n_cycles, 
                       use_fft = True,
                       return_itc = False, 
                       decim = 3, 
                       n_jobs = 1,
                       average = False)
    return power

ERROR! Session/line number was not unique in database. History logging moved to new session 146


In [6]:
# layout = BIDSLayout(BIDS_ROOT, derivatives = True)
# runs = layout.get_runs()
# for (fpath, sub, task, run) in get_fpaths(BIDS_ROOT):
#     print(fpath, sub, task, run)
# (fpaths, subs, tasks, runs) = get_fpaths(BIDS_ROOT)

In [20]:
# (fpaths, subjects, tasks, runs) = get_fpaths(BIDS_ROOT)

powers = []
events = []
for (fpath, sub, task, run) in get_fpaths(BIDS_ROOT):
    epochs = load_fif(fpath)
    epochs = epochs.crop(tmin = 0)
    events.append(mne.read_events(fpath))
    power = get_power_at_stim_freqs(epochs, STIM_FREQS)
    log_power = np.log10(power)
    powers.append(log_power)

Reading /Users/nusbaumlab/src/pitch_tracking/analysis/../data/bids/derivatives/preprocessing/sub-2/sub-2_task-pitch_run-1_desc-clean_epo.fif.gz ...


/Users/nusbaumlab/opt/anaconda3/envs/pitch-tracking/lib/python3.10/site-packages/bids/layout/validation.py:149: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


    Found the data of interest:
        t =    -200.00 ...     250.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
4553 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
Reading /Users/nusbaumlab/src/pitch_tracking/analysis/../data/bids/derivatives/preprocessing/sub-3/sub-3_task-pitch_run-1_desc-clean_epo.fif.gz ...
    Found the data of interest:
        t =    -200.00 ...     250.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
2333 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
Reading /Users/nusbaumlab/src/pitch_tracking/analysis/../data/bids/derivatives/preprocessing/sub-3/sub-3_task-pitch_run-2_desc-clean_epo.fif.gz ...
    Found the data of interest:
        t =    -200.00 ...     250.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
2327 

In [27]:
# power = powers[0]
# power.plot()
for i in range(len(powers)):
    print(np.shape(powers[i]))
# np.shape(powers[])
# np.shape(powers[1])
len(powers)

(4553, 62, 5, 167)
(2333, 62, 5, 167)
(2327, 62, 5, 167)
(2417, 61, 5, 167)
(1921, 61, 5, 167)
(4730, 62, 5, 167)
(4774, 62, 5, 167)
(4553, 62, 5, 167)
(4553, 62, 5, 167)


9

In [15]:
# np.shape(events)

(4774, 3)

In [4]:
# # layout = BIDSLayout(BIDS_ROOT, derivatives = True)
# # runs = layout.get_runs()
# for (fpath, sub, task, run) in get_fpaths(BIDS_ROOT):
#     print(fpath, sub, task, run)
# # (fpaths, subs, tasks, runs) = get_fpaths(BIDS_ROOT)

/Users/nusbaumlab/src/pitch_tracking/analysis/../data/bids/derivatives/preprocessing/sub-2/sub-2_task-pitch_run-1_desc-clean_epo.fif.gz 2 tasks 1
/Users/nusbaumlab/src/pitch_tracking/analysis/../data/bids/derivatives/preprocessing/sub-3/sub-3_task-pitch_run-1_desc-clean_epo.fif.gz 3 tasks 1
/Users/nusbaumlab/src/pitch_tracking/analysis/../data/bids/derivatives/preprocessing/sub-3/sub-3_task-pitch_run-2_desc-clean_epo.fif.gz 3 tasks 2
/Users/nusbaumlab/src/pitch_tracking/analysis/../data/bids/derivatives/preprocessing/sub-4/sub-4_task-pitch_run-1_desc-clean_epo.fif.gz 4 tasks 1
/Users/nusbaumlab/src/pitch_tracking/analysis/../data/bids/derivatives/preprocessing/sub-4/sub-4_task-pitch_run-2_desc-clean_epo.fif.gz 4 tasks 2
/Users/nusbaumlab/src/pitch_tracking/analysis/../data/bids/derivatives/preprocessing/sub-5/sub-5_task-pitch_run-1_desc-clean_epo.fif.gz 5 tasks 1
/Users/nusbaumlab/src/pitch_tracking/analysis/../data/bids/derivatives/preprocessing/sub-6/sub-6_task-pitch_run-1_desc-clean

/Users/nusbaumlab/opt/anaconda3/envs/pitch-tracking/lib/python3.10/site-packages/bids/layout/validation.py:149: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


In [ ]:
# import re
# import itertools
# layout = BIDSLayout(BIDS_ROOT, derivatives = True)
# fpaths = layout.get(scope = 'preprocessing',
#                     extension = 'fif.gz',
#                     return_type = 'filename')
# filter_runs = re.compile('run-(\d)')
# runs = list(map(filter_runs.findall, fpaths))
# runs = list(itertools.chain(*runs))
# runs
tup = ([1, 2, 3], [4, 5, 6])
for i

In [ ]:
for (fpath, sub, task, run) in get_fpaths(BIDS_ROOT):
    epochs = load_fif(fpath)
    epochs = epochs.crop(tmin = 0)
    n_cycles = STIM_FREQS / 7.  # different number of cycle per frequency
    print("Getting power")
    avg_power = tfr_morlet(epochs, 
                       freqs = STIM_FREQS, 
                       n_cycles = n_cycles, 
                       use_fft = True,
                       return_itc = True, 
                       decim = 3, 
                       n_jobs = 1,
                       average = False,
                       verbose = True)
    avg_log_power = np.log10(avg_power)
    break


# power.plot_topo(baseline=(-0.5, 0), title='Average power')
# power.plot([82], baseline=(-0.5, 0), mode='logratio', title=power.ch_names[82])

Reading /Users/nusbaumlab/src/pitch_tracking/analysis/../data/bids/derivatives/preprocessing/sub-2/sub-2_task-pitch_run-1_desc-clean_epo.fif.gz ...


/Users/nusbaumlab/opt/anaconda3/envs/pitch-tracking/lib/python3.10/site-packages/bids/layout/validation.py:149: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


    Found the data of interest:
        t =    -200.00 ...     250.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
4553 matching events found
No baseline correction applied
0 projection items activated
Getting power
